In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
df = pd.read_csv('thread_df/thread_df_1_9.csv', index_col=0)
comments = df.dropna(subset=['comment'])

In [16]:
sia = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    return sia.polarity_scores(text)['compound']

In [17]:
comments['sentiment_score'] = comments['comment'].apply(get_sentiment_score)

C:\Users\franc\AppData\Local\Temp\ipykernel_25176\3215188887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['sentiment_score'] = comments['comment'].apply(get_sentiment_score)


In [30]:
import re
def mark_relevant(dfi):
    df = dfi.copy()
    df['sold_msg'] = df.comment.str.match(pat = '^Sold to .*')
    if df['sold_msg'].astype(int).sum() == 0:
        df['sold_msg'] = df.comment.str.match(pat = '^Reserve not met.*')
    sold_pos = df.loc[df['sold_msg'] == True].iloc[0,0]
    df['relevant'] = df['position'] >= sold_pos
    df.drop(columns = ['sold_msg'], inplace = True)
    return df

In [31]:
test = mark_relevant(comments)
relevant_comments = test[test['relevant'] == True].iloc[1:] # get rid of irrelevant comments after car selling
final_sentiment = relevant_comments.sentiment_score.sum()/len(relevant_comments)

no soldm sg


In [33]:
relevant_comments

,position,user,comment,bid,upvote,rep,seller,bidder,verified,sentiment_score,relevant
3,4,bitransit,"Re: Nomodsplease\nYes, the multi contour seats...",NaN,4\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.4019,True
4,5,Nomodsplease,Do the multi-contour seats work? In other word...,NaN,1\nReply\nFlag as inappropriate,Reputation Icon10,0,0,0,0.0000,True
6,7,bitransit,"Re: salehi\nHere is the driving link to view,\...",NaN,3\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.0000,True
9,10,bitransit,Re: salehi\nHello. The cold start video has be...,NaN,3\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.2732,True
10,11,salehi,Would you be able to post a video that shows a...,NaN,2\nReply\nFlag as inappropriate,Reputation Icon31,0,1,1,0.3182,True
14,15,my3swedes,I love seeing these when they’ve been loved an...,NaN,1\nReply\nFlag as inappropriate,Reputation Icon535,0,0,1,0.9552,True
16,17,bitransit,"Hi All, it's Vaughn here to answer any questio...",NaN,1\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.6114,True


In [13]:
all_files = glob.glob(os.path.join('thread_df', '*.csv'))

In [14]:
df_out = pd.DataFrame(columns = ['id', 'sentiment_score'])

for file in all_files:
    df = pd.read_csv(file, index_col=0)
    id = file.replace('thread_df\\thread_df_', '').replace('.csv','')
    comments = df.dropna(subset = ['comment'])
    comments.loc[:,'sentiment_score'] = comments['comment'].apply(get_sentiment_score)
    test = mark_relevant(comments)
    relevant_comments = test[test['relevant'] == True].iloc[1:]
    final_sentiment = relevant_comments.sentiment_score.sum()/len(relevant_comments)
    df_out.loc[len(df_out.index)] = [id, final_sentiment]

c:\Users\franc\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\franc\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
c:\Users\franc\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

IndexError: single positional indexer is out-of-bounds